In [14]:
import nltk
import spacy
import string
import gensim

from gensim import corpora
from pprint import pprint
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from spacy.lang.da.stop_words import STOP_WORDS
from spacy.lang.da.examples import sentences 
from nltk.stem.snowball import DanishStemmer

In [2]:
#conda install -c conda-forge spacy-model-da_core_news_sm


### First example i have choosen i KMD and and article about them.

#### Starting with loading in the data

In [3]:
with open('kmd.txt', 'r') as f:
    text = f.read()

In [4]:
text

'Vindmølleproducenten skifter sine 6.000 danske ansatte til den cloudbaserede KMD Payroll Cloud.\n”KMD Payroll Cloud passer perfekt til os, da løsningen optimerer en række processer, og samtidig vil KMD håndtere mange administrationsopgaver. Dermed får vi en delvist outsourcet løsning, hvor vi stadig selv er i kontrol med vores egne data,” siger Kirsten Therkildsen, som er Head of HR Services i Siemens Gamesa.\n”Nu glæder vi os bare til at komme i gang med projektet, og vi ser meget frem til samarbejdet med KMD," tilføjer hun.\nOgså i KMD er der glæde over aftalen. \n”Vi er stolte af, at Siemens Gamesa valgte os som deres partner. Samtidig får vi nu en ny kunde ind, hvor vi integrerer KMD Payroll Cloud med virksomhedens HR-system – Workday – så vi dækker efterhånden integrationen til de førende globale HR-systemer. Det giver nogle gode markedsmæssige muligheder,” siger Kristian de Linde, Business Line Director i KMD.\n”I hele forløbet har vi haft en meget tæt kontakt til kunden – vi ha

# Tokenize the text

In [5]:
tokens = word_tokenize(text)

In [13]:
#tokens

# Remove Stop Words in "Danish"

In [7]:
stop_words = set(stopwords.words('danish'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

In [18]:
filtered_tokens

['Vindmølleproducenten',
 'skifter',
 '6.000',
 'danske',
 'ansatte',
 'cloudbaserede',
 'KMD',
 'Payroll',
 'Cloud',
 '.',
 '”',
 'KMD',
 'Payroll',
 'Cloud',
 'passer',
 'perfekt',
 ',',
 'løsningen',
 'optimerer',
 'række',
 'processer',
 ',',
 'samtidig',
 'KMD',
 'håndtere',
 'administrationsopgaver',
 '.',
 'Dermed',
 'får',
 'delvist',
 'outsourcet',
 'løsning',
 ',',
 'stadig',
 'kontrol',
 'vores',
 'egne',
 'data',
 ',',
 '”',
 'siger',
 'Kirsten',
 'Therkildsen',
 ',',
 'Head',
 'of',
 'HR',
 'Services',
 'Siemens',
 'Gamesa',
 '.',
 '”',
 'glæder',
 'bare',
 'komme',
 'gang',
 'projektet',
 ',',
 'ser',
 'frem',
 'samarbejdet',
 'KMD',
 ',',
 "''",
 'tilføjer',
 '.',
 'KMD',
 'glæde',
 'aftalen',
 '.',
 '”',
 'stolte',
 ',',
 'Siemens',
 'Gamesa',
 'valgte',
 'partner',
 '.',
 'Samtidig',
 'får',
 'ny',
 'kunde',
 ',',
 'integrerer',
 'KMD',
 'Payroll',
 'Cloud',
 'virksomhedens',
 'HR-system',
 '–',
 'Workday',
 '–',
 'så',
 'dækker',
 'efterhånden',
 'integrationen',
 'fø

# Lemmatize the tokens

#### So that i can reduce the words to their root form, where i use WordNetLemmatizer

In [8]:
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

# Part of Speech Tagging

#### Trying to implement POS tagging which could be helpful to identify the role of each word in the text

#### NNP	proper noun, singular - VBD verb past tense - CD cardinal digit - NN noun, singular (cat, tree and so on) - VB verb (ask) - JJ - This NLTK POS Tag is an adjective (large)
#### the rest can be found here: https://www.guru99.com/pos-tagging-chunking-nltk.html

In [9]:
pos_tagged_tokens = nltk.pos_tag(lemmatized_tokens)

In [24]:
pos_tagged_tokens

[('Vindmølleproducenten', 'NNP'),
 ('skifter', 'VBD'),
 ('6.000', 'CD'),
 ('danske', 'NN'),
 ('ansatte', 'NN'),
 ('cloudbaserede', 'NN'),
 ('KMD', 'NNP'),
 ('Payroll', 'NNP'),
 ('Cloud', 'NNP'),
 ('.', '.'),
 ('”', 'VB'),
 ('KMD', 'NNP'),
 ('Payroll', 'NNP'),
 ('Cloud', 'NNP'),
 ('passer', 'NN'),
 ('perfekt', 'NN'),
 (',', ','),
 ('løsningen', 'NN'),
 ('optimerer', 'NN'),
 ('række', 'NN'),
 ('processer', 'NN'),
 (',', ','),
 ('samtidig', 'JJ'),
 ('KMD', 'NNP'),
 ('håndtere', 'RB'),
 ('administrationsopgaver', 'RB'),
 ('.', '.'),
 ('Dermed', 'NNP'),
 ('får', 'JJ'),
 ('delvist', 'NN'),
 ('outsourcet', 'IN'),
 ('løsning', 'NN'),
 (',', ','),
 ('stadig', 'VB'),
 ('kontrol', 'NN'),
 ('vores', 'NNS'),
 ('egne', 'VBP'),
 ('data', 'NNS'),
 (',', ','),
 ('”', 'NNP'),
 ('siger', 'NN'),
 ('Kirsten', 'NNP'),
 ('Therkildsen', 'NNP'),
 (',', ','),
 ('Head', 'NNP'),
 ('of', 'IN'),
 ('HR', 'NNP'),
 ('Services', 'NNPS'),
 ('Siemens', 'NNP'),
 ('Gamesa', 'NNP'),
 ('.', '.'),
 ('”', 'NN'),
 ('glæder', 'N

### Named Entity Recognition where im trying ti identify entities in the text

### this have been trained to recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

In [10]:
nlp = spacy.load('da_core_news_sm')
doc = nlp(text)
entities = [(entity.text, entity.label_) for entity in doc.ents]

In [16]:
entities

[('danske', 'MISC'),
 ('KMD Payroll Cloud', 'ORG'),
 ('KMD', 'ORG'),
 ('KMD', 'ORG'),
 ('Kirsten Therkildsen', 'PER'),
 ('Siemens Gamesa', 'LOC'),
 ('KMD', 'ORG'),
 ('KMD', 'ORG'),
 ('Siemens Gamesa', 'PER'),
 ('KMD Payroll Cloud', 'ORG'),
 ('HR-system', 'MISC'),
 ('HR-systemer', 'MISC'),
 ('Kristian de Linde', 'PER'),
 ('Business Line Director', 'PER'),
 ('KMD', 'ORG'),
 ('Siemens Gamesa', 'LOC'),
 ('Kristian de Linde', 'PER'),
 ('Siemens Gamesa', 'PER'),
 ('KMD Nettoløn', 'ORG'),
 ('KMD Nettoløn', 'ORG'),
 ('KMD Payroll Cloud', 'ORG')]

### Making a function that is able to do everything as above, and then just return the text

In [7]:
# Loading Danish language model in spaCy
nlp = spacy.load('da_core_news_sm')
stemmer = DanishStemmer()

def preprocess(doc):
    # Tokenizinfdg the  text and removing punctuation
    tokens = [token.text for token in nlp(doc) if not token.is_punct]
    
    # Remove alle the stop words, lemmatize tokens, and stem tokens
    tokens = [stemmer.stem(token) for token in tokens if token.lower() not in STOP_WORDS]
    
    # Tag tokens with parts of speech
    pos_tags = [token.pos_ for token in nlp(' '.join(tokens))]
    
    # Only keep nouns and adjectives
    tokens = [token for i, token in enumerate(tokens) if pos_tags[i] in ['NOUN', 'ADJ']]
    
    # Join the tokens into a single string
    filtered_text = ' '.join(tokens)
    return filtered_text

document = 'Vindmølleproducenten skifter sine 6.000 danske ansatte til den cloudbaserede KMD Payroll Cloud.\n”KMD Payroll Cloud passer perfekt til os, da løsningen optimerer en række processer, og samtidig vil KMD håndtere mange administrationsopgaver. Dermed får vi en delvist outsourcet løsning, hvor vi stadig selv er i kontrol med vores egne data,” siger Kirsten Therkildsen, som er Head of HR Services i Siemens Gamesa.\n”Nu glæder vi os bare til at komme i gang med projektet, og vi ser meget frem til samarbejdet med KMD," tilføjer hun.\nOgså i KMD er der glæde over aftalen. \n”Vi er stolte af, at Siemens Gamesa valgte os som deres partner. Samtidig får vi nu en ny kunde ind, hvor vi integrerer KMD Payroll Cloud med virksomhedens HR-system – Workday – så vi dækker efterhånden integrationen til de førende globale HR-systemer. Det giver nogle gode markedsmæssige muligheder,” siger Kristian de Linde, Business Line Director i KMD.\n”I hele forløbet har vi haft en meget tæt kontakt til kunden – vi har lyttet, forstået kundens behov og i sidste ende tilbudt en løsning, som passer til Siemens Gamesa,” siger Kristian de Linde og sender en stor tak til teamet bag den nye aftale.\nSiemens Gamesa anvender i dag løsningen KMD Nettoløn og er den første af de mange kunder på KMD Nettoløn, som bliver løftet til den cloudbaserede KMD Payroll Cloud.'
preprocessed_doc = preprocess(document)
print(preprocessed_doc)

vindmølleproducent skift dansk cloudbas payrol kmd payrol løsning optim proces samtid hånd outsourc løsning kontrol egn data therkilds glæd gang kmd aftal stolt siem valg partn samtid kund integr payrol virksom hr-system workday dæk efterhånd integration global hr-system god markedsmæs lin director kmd hel forløb kontak kund behov tilbud løsning stor tak team nye aftal anvend dag løsning nettoløn kund cloudbas kmd payrol


In [8]:
nlp = spacy.blank("da")
nlp.add_pipe("sentencizer")
nlp.add_pipe("asent_da_v1")
doc = nlp(preprocessed_doc)
print(doc._.polarity)

neg=0.0 neu=0.733 pos=0.267 compound=0.9517


### In this example we can again use Asent to obtain more information and in this situation the rated valence of a single token. We can see that both "stolt" and "god" is rated postive

In [9]:
for token in doc:
    print(f"{token._.polarity} | Valence: {token._.valence} | Negation: {token._.is_negation}")

polarity=0.0 token=vindmølleproducent span=vindmølleproducent | Valence: 0.0 | Negation: False
polarity=0.0 token=skift span=skift | Valence: 0.0 | Negation: False
polarity=0.0 token=dansk span=dansk | Valence: 0.0 | Negation: False
polarity=0.0 token=cloudbas span=cloudbas | Valence: 0.0 | Negation: False
polarity=0.0 token=payrol span=payrol | Valence: 0.0 | Negation: False
polarity=0.0 token=kmd span=kmd | Valence: 0.0 | Negation: False
polarity=0.0 token=payrol span=payrol | Valence: 0.0 | Negation: False
polarity=1.0 token=løsning span=løsning | Valence: 1.0 | Negation: False
polarity=0.0 token=optim span=optim | Valence: 0.0 | Negation: False
polarity=0.0 token=proces span=proces | Valence: 0.0 | Negation: False
polarity=0.0 token=samtid span=samtid | Valence: 0.0 | Negation: False
polarity=0.0 token=hånd span=hånd | Valence: 0.0 | Negation: False
polarity=0.0 token=outsourc span=outsourc | Valence: 0.0 | Negation: False
polarity=1.0 token=løsning span=løsning | Valence: 1.0 | Ne

In [10]:
# visualize model prediction
asent.visualize(doc, style="prediction")

## Im using Latent Dirichlet Allocation trying to identify the main topics or themes discussed in the article. This can give me some ideas what the company is focused on and what they specialize in

### the output that i get is only based on a small text which isn't perfect, and in real-world scenarios, i would need to process a larger text corpus for the LDA model to get better results.

In [20]:
# Tokeniz the text
tokens = gensim.utils.simple_preprocess(preprocessed_doc)

# Creating a dictionary from the tokens
dictionary = corpora.Dictionary([tokens])

# Creating a corpus from the tokens
corpus = [dictionary.doc2bow([token]) for token in tokens]

# Define the number of topics
num_topics = 1

# Training tthe LDA model
lda_model = gensim.models.LdaModel(corpus=corpus,
                                   id2word=dictionary,
                                   num_topics=num_topics,
                                   random_state=100,
                                   chunksize=100,
                                   passes=10)
pprint(lda_model.print_topics())

[(0,
  '0.045*"løsning" + 0.045*"kmd" + 0.045*"payrol" + 0.036*"kund" + '
  '0.027*"system" + 0.027*"samtid" + 0.027*"aftal" + 0.027*"hr" + '
  '0.027*"cloudbas" + 0.018*"nye"')]
